In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize
import datetime

In [ ]:
data = np.loadtxt(open("../data/Vela_Flux.txt", 'rb'), usecols=range(7))

# This is how we pull out the data from columns in the array.

# This is the date in "Mission Elapsesd Time"
# For the Fermi mission, this is defined to be the number of seconds since the start of 2001.
date_MET = data[:,0]
# This is the offset in seconds between the Fermi "MET" and the UNIX "epoch" used by matplotlib
MET_To_Unix = 978336000

# These are the numbers of photons observed from Vela each week in the "low" Energy Band (100 MeV - 800 MeV)
nObs_LE = data[:,1]

# These are the number of photons expected from Vela each week, under the assumption that it is 
# not varying at all, and the only differences depend on how long we spent looking at Vela
# that particular weeek
nExp_LE = data[:,2]

# These are the band bounds, in MeV
LE_bounds = (100., 800.)

# This is the "significance" of the variation for each week.  We will discuss this more later
signif_LE = data[:,3]

nObs_HE = data[:,4]
nExp_HE = data[:,5]
signif_HE = data[:6]
HE_bounds = (800., 10000.)

# This converts the dates to something that matplotlib understands
dates = [datetime.datetime.fromtimestamp(date + MET_To_Unix) for date in date_MET]
date_YEAR = 2001 +  (date_MET / (24*3600*365))
years_since_mid_2014 = date_YEAR  - 2014.5

In [ ]:
excess_counts = nObs_LE-nExp_LE
sigma_counts =  np.sqrt(nObs_LE)
_ = plt.errorbar(dates, excess_counts, yerr=sigma_counts)
_ = plt.xlabel(r"Date [year]]")
_ = plt.ylabel(r"$n_{\rm obs}$ [per week]")

In [ ]:
_ = plt.errorbar(years_since_mid_2014, excess_counts, yerr=sigma_counts)
_ = plt.xlabel(r"Date [year]]")
_ = plt.ylabel(r"$n_{\rm obs}$ [per week]")

In [ ]:
def linear_function(xvals, params):
    return params[0] + xvals*params[1]

In [ ]:
_ = plt.xlabel(r"Date [year]]")
_ = plt.ylabel(r"$n_{\rm obs}$ [per week]")
_ = plt.errorbar(years_since_mid_2014, excess_counts, yerr=sigma_counts, alpha=0.2)

xvals = years_since_mid_2014
params = np.array([0, 0])
for offset in np.linspace(-200, 200, 5):
    params[0] = offset
    _ = plt.plot(xvals, linear_function(xvals, params), label=r"Offset = %0.0f" % offset)
_ = plt.legend()

In [ ]:
_ = plt.xlabel(r"Date [year]]")
_ = plt.ylabel(r"$n_{\rm obs}$ [per week]")
_ = plt.errorbar(years_since_mid_2014, excess_counts, yerr=sigma_counts, alpha=0.2)

xvals = years_since_mid_2014
params = np.array([0, 0])
for slope in np.linspace(-5, 5, 5):
    params[1] = slope
    _ = plt.plot(xvals, linear_function(xvals, params), label=r"Slope = %0.1f" % slope)
_ = plt.legend()

In [ ]:
def residual_function(data_x, data_y, model_function, params):
    model_y = model_function(data_x, params)
    residual = data_y - model_y
    return residual

In [ ]:
def plot_residuals(params):
    _ = plt.xlabel(r"Date [year]]")
    _ = plt.ylabel(r"$n_{\rm obs}$ [per week]")
    residuals = residual_function(years_since_mid_2014, excess_counts, linear_function, params)
    _ = plt.scatter(years_since_2008, residuals)

In [ ]:
plot_residuals([0,0])

In [ ]:
plot_residuals([0,10])

In [ ]:
def plot_scaled_residuals(params):
    residuals = residual_function(years_since_mid_2014, excess_counts, linear_function, params)
    chi = residuals / sigma_counts
    _ = plt.xlabel(r"Date [year]]")
    _ = plt.ylabel(r"$n_{\rm obs}$ [per week]")
    _ = plt.scatter(years_since_2008, chi)


In [ ]:
plot_scaled_residuals([0,10])

In [ ]:
def chi2_function(data_x, data_y, data_sigma_y, model_function, params):
    model_y = model_function(data_x, params)
    chi2 = ((data_y - model_y)/(data_sigma_y))**2
    return chi2.sum()

In [ ]:
_ = plt.xlabel(r"Offest")
_ = plt.ylabel(r"$\chi^2$")

params = np.array([0, 0])
chi2_vals = np.zeros((11))
offset_scan_points = np.linspace(-20., 20., 11)
for i in range(11):
    params[0] = offset_scan_points[i]
    chi2_vals[i] = chi2_function(years_since_mid_2014, excess_counts, sigma_counts, linear_function, params)

_ = plt.plot(offset_scan_points, chi2_vals)

In [ ]:
_ = plt.xlabel(r"Offest")
_ = plt.ylabel(r"$\chi^2$")

params = np.array([0., 0.])
chi2_vals = np.zeros((11))
offset_scan_points = np.linspace(-10., 10., 11)
for i in range(11):
    params[0] = offset_scan_points[i]
    chi2_vals[i] = chi2_function(years_since_mid_2014, excess_counts, sigma_counts, linear_function, params)

_ = plt.plot(offset_scan_points, chi2_vals)

In [ ]:
_ = plt.xlabel(r"Slope")
_ = plt.ylabel(r"$\chi^2$")

params = np.array([0., 0.])
chi2_vals = np.zeros((11))
slope_scan_points = np.linspace(-3., 3., 11)
for i in range(11):
    params[1] = slope_scan_points[i]
    chi2_vals[i] = chi2_function(years_since_mid_2014, excess_counts, sigma_counts, linear_function, params)

_ = plt.plot(slope_scan_points, chi2_vals)

In [ ]:
_ = plt.ylabel(r"Slope")
_ = plt.xlabel(r"Offset")

nx = 51
ny = 51

params = np.array([0., 0.])
chi2_2d_scan_vals = np.zeros((nx, ny))
offset_scan_points = np.linspace(-10, 10, nx)
slope_scan_points = np.linspace(-5, 5, ny)

for i in range(nx):
    params[0] = offset_scan_points[i]
    for j in range(ny):
        params[1] = slope_scan_points[j]
        chi2_2d_scan_vals[i,j] = chi2_function(years_since_mid_2014, excess_counts, sigma_counts,
                                               linear_function, params)

chi2_2d_scan_vals -= chi2_2d_scan_vals.min()
_ = plt.imshow(chi2_2d_scan_vals.T, extent=(-10, 10, -5, 5), aspect='auto')
_ = plt.colorbar()
_ = plt.contour(offset_scan_points, slope_scan_points, chi2_2d_scan_vals.T, levels=[0.5, 2, 4.5])

In [ ]:
def cost_function(params):
    return chi2_function(years_since_mid_2014, excess_counts, sigma_counts, linear_function, params)

In [ ]:
cost_function(np.array([0., 0.]))

In [ ]:
init_pars = np.array([0., 0.])
result = optimize.minimize(cost_function, x0=np.array(init_pars))

In [ ]:
fit_cov = result['hess_inv']
best_fit = result['x']
offset_fit = best_fit[0]
slope_fit = best_fit[1]
offset_error = np.sqrt(fit_cov[0,0])
slope_error = np.sqrt(fit_cov[1,1])
offset_slope_correl = fit_cov[0,1]/(offset_error*slope_error)
print("Best Fit:")
print("Offset = %0.1f +- %0.1f" % (offset_fit, offset_error))
print("Slope = %0.2f +- %0.2f" % (slope_fit, slope_error))
print("Correlation = %0.2f" % offset_slope_correl)

In [ ]:
_ = plt.ylabel(r"Slope")
_ = plt.xlabel(r"Offset")

nx = 51
ny = 51

params = np.array([0., 0.])
chi2_2d_scan_vals = np.zeros((nx, ny))
offset_scan_points = np.linspace(-5, 5, nx)
slope_scan_points = np.linspace(-2, 2, ny)

for i in range(nx):
    params[0] = offset_scan_points[i]
    for j in range(ny):
        params[1] = slope_scan_points[j]
        chi2_2d_scan_vals[i,j] = chi2_function(years_since_mid_2014, excess_counts, sigma_counts, linear_function, params)

chi2_2d_scan_vals -= best_fit_chi2
_ = plt.imshow(chi2_2d_scan_vals.T, extent=(-5, 5, -2, 2), aspect='auto')
_ = plt.colorbar()
_ = plt.contour(offset_scan_points, slope_scan_points, chi2_2d_scan_vals.T, levels=[0.5, 2, 4.5])
_ = plt.errorbar(offset_fit, slope_fit, xerr=offset_error, yerr=slope_error)